In [1]:
from datetime import datetime
import requests, json, os
from typing import Dict, Annotated, Optional, List, Union
from dotenv import load_dotenv
load_dotenv()

from services.twilio import client
from services.cache import get_agent_metadata, get_all_agents
from services.chat.chat import similarity_search, get_embedding
from services.db.supabase_services import supabase_client
supabase = supabase_client()

from app.core.config import settings

In [ ]:
""" Get all connected apps and accounts """
""" store connection ID to each user_id in supabase """
from composio import Composio

# Initialize the Composio client
client = Composio()

# Get the list of integrations (connected apps)
integrations = client.integrations.get()
print("Connected Apps:")
for integration in integrations:
    print(f"• App: {integration.appName}")
    print(f"  ID: {integration.id}")

# Get the list of connected accounts (existing connections)
connected_accounts = client.connected_accounts.get()
print("\nExisting Connections:")
for account in connected_accounts:
    print(f"• ID: {account.id}")
    print(f"  App: {account.appName}")
    print(f"  Status: {account.status}")


In [40]:
from composio import Composio

# Initialize the Composio client
composio_client = Composio()

# Create a new entity with a custom ID
new_entity = composio_client.get_entity(id="user_2mmXezcGmjZCf88gT2v2waCBsXv")


In [39]:
""" checks if entity already exists """

existing_entity = composio_client.get_entity(id="user_2mmXezcGmjZCf88gT2v2waCBsXv")
existing_connections = existing_entity.get_connections()

if existing_connections:
    print("Entity already exists with connections:", existing_connections)
else:
    print("Entity is newly created or has no connections")


# # Check for connected accounts for the new entity
# try:
#     connected_account = new_entity.get_connection(app="outlook")
#     print(f"Connected Outlook account: {connected_account}")
# except Exception as e:
#     print("No connected Slack account found")

# # Perform actions using the new entity
# actions = new_entity.get_tools(app_name="github")



Entity already exists with connections: [ConnectedAccountModel(id='7ede5a6d-acf7-4b6d-9426-13dc1a4616d3', status='ACTIVE', createdAt='2024-11-24T02:13:42.541Z', updatedAt='2024-11-24T02:13:52.288Z', appUniqueId='notion', appName='notion', integrationId='f9721737-acc8-42f9-ae18-f997370abf47', connectionParams=AuthConnectionParamsModel(scope=None, base_url='********', client_id='********', token_type='bearer', access_token='********', client_secret='********', consumer_id=None, consumer_secret=None, headers=None, queryParams=None), clientUniqueUserId='user_2mmXezcGmjZCf88gT2v2waCBsXv', entityId=None)]


### Outlook

In [43]:
""" this function will be triggered in endpoint /composion/new_connection """
""" then to store the connection ID to the user_id in supabase """
""" frontend to have button to "generate auth link", response from this endpoint will be the redirect url """
### entity ID will be created for each user. 
""" {"app": "outlook", "connectedAccountId" : "xxx", "client_id" : "xxx" } """

"""Auth"""
from composio import ComposioToolSet, App
from composio import Composio

# Initialize the Composio client
composio_client = Composio()

# # Create a new entity with a custom ID
# new_entity = composio_client.get_entity(id="user_2mmXezcGmjZCf88gT2v2waCBsXv")


toolset = ComposioToolSet()
entity = toolset.get_entity(id="user_2mmXezcGmjZCf88gT2v2waCBsXvww")



# request = entity.initiate_connection(App.OUTLOOK)
# ### background task save request.connectedAccountId to integrations table in supabase 
# ### then return the below redirect url to frontend
# print(f"Open this URL to authenticate: {request.redirectUrl}")


[2024-11-24 02:43:10,788][INFO] Logging is set to INFO, use `logging_level` argument or `COMPOSIO_LOGGING_LEVEL` change this


In [44]:
entity.initiate_connection(App.OUTLOOK)

ConnectionRequestModel(connectionStatus='INITIATED', connectedAccountId='e9f2ecdf-24d9-4e32-a13e-ca539ae1fe9a', redirectUrl='https://login.microsoftonline.com/common/oauth2/v2.0/authorize?client_id=05a98fa5-dad5-44ea-ad93-d190ee828ee1&redirect_uri=https%3A%2F%2Fbackend.composio.dev%2Fapi%2Fv1%2Fauth-apps%2Fadd&scope=Calendars.Read+Calendars.ReadWrite+Contacts.ReadWrite+Mail.Read+Mail.ReadWrite+Mail.Send+offline_access+User.Read&response_type=code&response_mode=query&prompt=consent&state=production_eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJjb25uZWN0aW9uSWQiOiJlOWYyZWNkZi0yNGQ5LTRlMzItYTEzZS1jYTUzOWFlMWZlOWEiLCJpbnRlZ3JhdGlvbklkIjoiZjcwNzkzYmEtYjJlYS00N2MyLTgwNDQtYjQ5NTBjNWQ1ZTYxIiwiYXBwTmFtZSI6Im91dGxvb2siLCJjbGllbnRJbmZvIjp7ImNsaWVudElkIjoiMDEyNzE0NTktODYzZi00NzcwLWE4YTMtMTY2YWNiNzgxZjM3IiwibWVtYmVySWQiOiIyYmQyNjA1ZC0yYjk4LTQwZTgtYmM1Ni1mOGQ2NmExNTY5ZWMifSwiaWF0IjoxNzMyNDE2MTk1fQ.PFX8jg5uz3gupkth2-96pPn3vjZPTvuak2hWvXJeuLA&code_challenge=yB9VhMj4niW-8HL7LyBL5h8PCn3YM98Ln0PpDBF9AGg&code_c

In [20]:
from urllib.parse import urlparse, parse_qs

url = request.redirectUrl
parsed_url = urlparse(url)
query_params = parse_qs(parsed_url.query)
client_id = query_params.get('client_id', [None])[0]

request.connectedAccountId 

05a98fa5-dad5-44ea-ad93-d190ee828ee1


### Shopify

In [ ]:
"""Auth"""
""" shopify not working :("""

from composio import ComposioToolSet, App

toolset = ComposioToolSet()
entity = toolset.get_entity()
request = entity.initiate_connection(App.SHOPIFY)

print(f"Open this URL to authenticate: {request.redirectUrl}")


### Notion

In [38]:
"""Auth"""

from composio import ComposioToolSet, App

toolset = ComposioToolSet(entity_id="Jessica")
entity = toolset.get_entity()
request = entity.initiate_connection(App.NOTION)

print(f"Open this URL to authenticate: {request.redirectUrl}")


[2024-11-24 02:13:41,224][INFO] Logging is set to INFO, use `logging_level` argument or `COMPOSIO_LOGGING_LEVEL` change this


Open this URL to authenticate: https://api.notion.com/v1/oauth/authorize?client_id=17eb9b75-7ad0-4fc1-b01d-12bb09271484&redirect_uri=https%3A%2F%2Fbackend.composio.dev%2Fapi%2Fv1%2Fauth-apps%2Fadd&scope=&response_type=code&owner=user&state=production_eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJjb25uZWN0aW9uSWQiOiI3ZWRlNWE2ZC1hY2Y3LTRiNmQtOTQyNi0xM2RjMWE0NjE2ZDMiLCJpbnRlZ3JhdGlvbklkIjoiZjk3MjE3MzctYWNjOC00MmY5LWFlMTgtZjk5NzM3MGFiZjQ3IiwiYXBwTmFtZSI6Im5vdGlvbiIsImNsaWVudEluZm8iOnsiY2xpZW50SWQiOiIwMTI3MTQ1OS04NjNmLTQ3NzAtYThhMy0xNjZhY2I3ODFmMzciLCJtZW1iZXJJZCI6IjJiZDI2MDVkLTJiOTgtNDBlOC1iYzU2LWY4ZDY2YTE1NjllYyJ9LCJpYXQiOjE3MzI0MTQ0MjJ9.MGqznQrMQaONYyL-xMbFLE1bizfNc9Eh-76rEhpMWSU&code_challenge=QK1DSIfYRAQgkR_gpLfvqZFPbZFJnSAh1V5qR-ml2YE&code_challenge_method=S256


In [3]:
from composio_openai import ComposioToolSet, Action
from openai import OpenAI

openai_client = OpenAI()
composio_toolset = ComposioToolSet(entity_id="Jessica")


[2024-11-23 04:32:28,947][INFO] Logging is set to INFO, use `logging_level` argument or `COMPOSIO_LOGGING_LEVEL` change this


In [16]:
today = datetime.now().strftime("%Y-%m-%d")
task = "get the DB id for DevOps DB"

# Update the tools to include search capability
tools = composio_toolset.get_tools(actions=[
    Action.NOTION_SEARCH_NOTION_PAGE])

# First, let's search for your database
search_response = openai_client.chat.completions.create(
    model="gpt-4o",
    tools=tools,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Search for a database called DevOps DB in my Notion workspace"},
    ],
)
search_result = composio_toolset.handle_tool_calls(search_response)

db_id = search_result[0]['data']['response_data']['results'][0]['id']
print("DB ID:", db_id)

task = f"get the page to do's in DB with id {db_id}"

# Update the tools to include search capability
tools = composio_toolset.get_tools(actions=[
    Action.NOTION_QUERY_DATABASE])

search_response = openai_client.chat.completions.create(
    model="gpt-4o",
    tools=tools,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Search for a database called DevOps DB in my Notion workspace"},
    ],
)
search_result = composio_toolset.handle_tool_calls(search_response)



[2024-11-23 04:57:43,155][INFO] Executing `NOTION_SEARCH_NOTION_PAGE` with params={'query': 'DevOps DB', 'filter_value': 'database'} and metadata={} connected_account_id=None
[2024-11-23 04:57:46,528][INFO] Got response={'successfull': True, 'data': {'response_data': {'object': 'list', 'results': [{'object': 'database', 'id': '11b53e39-0487-804b-bca4-dd524767939b', 'cover': {'type': 'external', 'external': {'url': 'https://www.notion.so/images/page-cover/nasa...


DB ID: 11b53e39-0487-804b-bca4-dd524767939b


In [17]:
print(search_result)


[]


### Google Calendar

In [5]:

tools = composio_toolset.get_tools(actions=[Action.GOOGLECALENDAR_CREATE_EVENT])

task = "Create a 1 hour meeting event at 5:30PM tomorrow regarding OpenAI Integration"

today = datetime.now().strftime("%Y-%m-%d")

response = openai_client.chat.completions.create(
    model="gpt-4-turbo-preview",
    tools=tools,
    messages=[
        {"role": "system", "content": f"You are a helpful assistant. Today's date is {today}."},
        {"role": "user", "content": task},
    ],
)
result = composio_toolset.handle_tool_calls(response)
print(result)


[2024-11-23 04:40:44,386][INFO] Executing `GOOGLECALENDAR_CREATE_EVENT` with params={'calendar_id': 'primary', 'create_meeting_room': True, 'event_duration': '1h', 'start_datetime': '2024-11-24T17:30:00', 'summary': 'OpenAI Integration Meeting'} and metadata={} connected_account_id=None
[2024-11-23 04:40:46,334][INFO] Got response={'successfull': True, 'data': {'response_data': {'kind': 'calendar#event', 'etag': '"3464673691676000"', 'id': 'i62bupprdrf2veopdjo7l7629k', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=aTYyYnVwcHJkcmYydmVvcGRq...


[{'successfull': True, 'data': {'response_data': {'kind': 'calendar#event', 'etag': '"3464673691676000"', 'id': 'i62bupprdrf2veopdjo7l7629k', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=aTYyYnVwcHJkcmYydmVvcGRqbzdsNzYyOWsgbWljaGFlbEBmbG93b24uYWk', 'created': '2024-11-23T04:40:45.000Z', 'updated': '2024-11-23T04:40:45.838Z', 'summary': 'OpenAI Integration Meeting', 'creator': {'email': 'michael@flowon.ai', 'self': True}, 'organizer': {'email': 'michael@flowon.ai', 'self': True}, 'start': {'dateTime': '2024-11-24T17:30:00Z', 'timeZone': 'UTC'}, 'end': {'dateTime': '2024-11-24T18:30:00Z', 'timeZone': 'UTC'}, 'iCalUID': 'i62bupprdrf2veopdjo7l7629k@google.com', 'sequence': 0, 'hangoutLink': 'https://meet.google.com/egu-qfzk-xcg', 'conferenceData': {'createRequest': {'requestId': '0a06f1a31bcb4b8fb75d8144fc6848fb', 'conferenceSolutionKey': {'type': 'hangoutsMeet'}, 'status': {'statusCode': 'success'}}, 'entryPoints': [{'entryPointType': 'video', 'uri': 'http